In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyEnv

path = '../data/processed/'

### 1 - Input Data

In [2]:
categories = pd.read_csv(f'{path}categories.csv')
# Excedente de bicicletas disponíveis
a = categories.iloc[0].values
print('Disponibilidade: ', a)
# Espaço ocupado por cada categoria de bicicleta
s = categories.iloc[1].values
print('Capacidade: ', s)
# Capacidade total do caminhão
T = 80

areas = [pd.read_csv(f'{path}area{i+1}.csv') for i in range(7)]
matrix = np.stack([area.T.values for area in areas], axis=0)
print('Shape Áreas: ', matrix.shape)

Disponibilidade:  [272. 270. 279. 267. 282. 279.]
Capacidade:  [1.  1.5 1.5 1.7 1.7 4. ]
Shape Áreas:  (7, 6, 115)


### 2 - Model

In [3]:
model = pyEnv.ConcreteModel()

# índices para as áreas e categorias
model.I = pyEnv.Set(initialize=range(matrix.shape[0]))
model.J = pyEnv.Set(initialize=range(matrix.shape[1]))
model.K = pyEnv.Set(initialize=range(matrix.shape[2]))

# variáveis de decisão
model.x = pyEnv.Var(model.I, model.J, model.K, domain=pyEnv.Binary, initialize=0)

# função objetivo
model.profit = pyEnv.Objective(
    expr=sum(matrix[i, j, k] * model.x[i,j, k] for i in model.I for j in model.J for k in model.K),
    sense=pyEnv.maximize
)

# restrições
def supply_constraint(model, j):
    return sum(model.x[i, j, k] for i in model.I for k in model.K) <= a[j]

model.supply_constraint = pyEnv.Constraint(model.J, rule=supply_constraint)

def capacity_constraint(model):
    return sum(s[j] * model.x[i, j, k] for i in model.I for j in model.J for k in model.K) <= T

model.capacity_constraint = pyEnv.Constraint(rule=capacity_constraint)

# adicionar restrição condicional
def sequential_constraint_rule(model, i, j, k):
    if k == 0:
        return pyEnv.Constraint.Skip
    else:
        return model.x[i, j, k] <= model.x[i, j, k-1]

model.sequential_constraint = pyEnv.Constraint(model.I, model.J, model.K, rule=sequential_constraint_rule)


# resolver o modelo
solver = pyEnv.SolverFactory('glpk')
results = solver.solve(model)

# resultados
print("Status:", results.solver.termination_condition)
print("Valor objetivo ótimo (soma dos lucros esperados):", pyEnv.value(model.profit))

print("\nSolução ótima:")
for i in model.I:
    for j in model.J:
        for k in model.K:
            print(f"{i,j,k}: {model.x[i, j, k].value}")


Status: optimal
Valor objetivo ótimo (soma dos lucros esperados): 4358.783400000001

Solução ótima:
(0, 0, 0): 1.0
(0, 0, 1): 0.0
(0, 0, 2): 0.0
(0, 0, 3): 0.0
(0, 0, 4): 0.0
(0, 0, 5): 0.0
(0, 0, 6): 0.0
(0, 0, 7): 0.0
(0, 0, 8): 0.0
(0, 0, 9): 0.0
(0, 0, 10): 0.0
(0, 0, 11): 0.0
(0, 0, 12): 0.0
(0, 0, 13): 0.0
(0, 0, 14): 0.0
(0, 0, 15): 0.0
(0, 0, 16): 0.0
(0, 0, 17): 0.0
(0, 0, 18): 0.0
(0, 0, 19): 0.0
(0, 0, 20): 0.0
(0, 0, 21): 0.0
(0, 0, 22): 0.0
(0, 0, 23): 0.0
(0, 0, 24): 0.0
(0, 0, 25): 0.0
(0, 0, 26): 0.0
(0, 0, 27): 0.0
(0, 0, 28): 0.0
(0, 0, 29): 0.0
(0, 0, 30): 0.0
(0, 0, 31): 0.0
(0, 0, 32): 0.0
(0, 0, 33): 0.0
(0, 0, 34): 0.0
(0, 0, 35): 0.0
(0, 0, 36): 0.0
(0, 0, 37): 0.0
(0, 0, 38): 0.0
(0, 0, 39): 0.0
(0, 0, 40): 0.0
(0, 0, 41): 0.0
(0, 0, 42): 0.0
(0, 0, 43): 0.0
(0, 0, 44): 0.0
(0, 0, 45): 0.0
(0, 0, 46): 0.0
(0, 0, 47): 0.0
(0, 0, 48): 0.0
(0, 0, 49): 0.0
(0, 0, 50): 0.0
(0, 0, 51): 0.0
(0, 0, 52): 0.0
(0, 0, 53): 0.0
(0, 0, 54): 0.0
(0, 0, 55): 0.0
(0, 0, 56): 0.

In [4]:
sum_x = [[sum(model.x[i, j, k].value for k in model.K) for j in model.J] for i in model.I]
print('Quantidade de bicicletas alocadas da área i da categoria j:')
sum_x

Quantidade de bicicletas alocadas da área i da categoria j:


[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [69.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 [7.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [5]:
for i in model.I:
        for j in model.J:
            print(f"Número de bicicletas da categoria {j + 1} a serem movidas para a área {i + 1}: {sum_x[i][j]}")

Número de bicicletas da categoria 1 a serem movidas para a área 1: 1.0
Número de bicicletas da categoria 2 a serem movidas para a área 1: 0.0
Número de bicicletas da categoria 3 a serem movidas para a área 1: 0.0
Número de bicicletas da categoria 4 a serem movidas para a área 1: 0.0
Número de bicicletas da categoria 5 a serem movidas para a área 1: 0.0
Número de bicicletas da categoria 6 a serem movidas para a área 1: 0.0
Número de bicicletas da categoria 1 a serem movidas para a área 2: 0.0
Número de bicicletas da categoria 2 a serem movidas para a área 2: 0.0
Número de bicicletas da categoria 3 a serem movidas para a área 2: 0.0
Número de bicicletas da categoria 4 a serem movidas para a área 2: 0.0
Número de bicicletas da categoria 5 a serem movidas para a área 2: 0.0
Número de bicicletas da categoria 6 a serem movidas para a área 2: 0.0
Número de bicicletas da categoria 1 a serem movidas para a área 3: 69.0
Número de bicicletas da categoria 2 a serem movidas para a área 3: 1.0
Númer

### 3 - Function

In [6]:
import os
import sys
sys.path.append('..')

from src.solver import relocate_bicycles

profit, solution = relocate_bicycles(matrix, a, s, T)

Status: optimal
Valor objetivo ótimo (Lucro esperado): 4358.783400000001

Solução ótima:
Número de bicicletas da categoria 1 a serem movidas para a área 1: 1.0
Número de bicicletas da categoria 1 a serem movidas para a área 3: 69.0
Número de bicicletas da categoria 2 a serem movidas para a área 3: 1.0
Número de bicicletas da categoria 1 a serem movidas para a área 4: 7.0
Número de bicicletas da categoria 2 a serem movidas para a área 6: 1.0


In [7]:
profit

4358.783400000001

In [8]:
solution

array([[ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [69.,  1.,  0.,  0.,  0.,  0.],
       [ 7.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.]])

In [9]:
areas[2]['Child'][:69].sum() + areas[2]['Adult'][:1].sum() + areas[3]['Child'][:7].sum() + areas[0]['Child'][:1].sum() + areas[5]['Adult'][:1].sum()

4358.7834

In [12]:
profit, solution = relocate_bicycles(matrix, a, s, 10e12)

Status: optimal
Valor objetivo ótimo (Lucro esperado): 78866.16080000003

Solução ótima:
Número de bicicletas da categoria 2 a serem movidas para a área 1: 6.0
Número de bicicletas da categoria 1 a serem movidas para a área 3: 91.0
Número de bicicletas da categoria 2 a serem movidas para a área 3: 92.0
Número de bicicletas da categoria 3 a serem movidas para a área 3: 110.0
Número de bicicletas da categoria 4 a serem movidas para a área 3: 93.0
Número de bicicletas da categoria 5 a serem movidas para a área 3: 104.0
Número de bicicletas da categoria 6 a serem movidas para a área 3: 106.0
Número de bicicletas da categoria 1 a serem movidas para a área 4: 103.0
Número de bicicletas da categoria 2 a serem movidas para a área 4: 97.0
Número de bicicletas da categoria 3 a serem movidas para a área 4: 72.0
Número de bicicletas da categoria 4 a serem movidas para a área 4: 103.0
Número de bicicletas da categoria 5 a serem movidas para a área 4: 108.0
Número de bicicletas da categoria 6 a sere

In [13]:
solution

array([[  0.,   6.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [ 91.,  92., 110.,  93., 104., 106.],
       [103.,  97.,  72., 103., 108., 109.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [ 78.,  75.,  97.,  71.,  70.,  64.],
       [  0.,   0.,   0.,   0.,   0.,   0.]])